In [305]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats as stats
import statsmodels.api as sm
import sys
from datetime import datetime
import time

In [306]:
def accident_prob_front(reaction_time,cars_per_day):
    seconds_per_year = 86400 * 365
    danger_time_per_car = reaction_time
    cars_per_year = cars_per_day * 365
    danger_time_total = danger_time_per_car * cars_per_year
    danger_prob = (danger_time_total / seconds_per_year)

    return danger_prob

In [307]:
def net_test(net_status, weight_in_net, impact_energy):
    if net_status == True:
        return True
    elif weight_in_net > 2000:
        if impact_energy > 500:
            return True
        else:
            return False
    elif weight_in_net <= 2000:
        if impact_energy > 1000:
            return True
        else:
            return False
    else:
        return False

In [308]:
def accident_prob_top(v_km_h, laenge_auto, cars_per_day):
    seconds_per_year = 86400 * 365
    v_m_s = (v_km_h / 3.6)
    danger_time_per_car = (laenge_auto / v_m_s)
    cars_per_year = cars_per_day * 365
    danger_time_total = danger_time_per_car * cars_per_year
    danger_prob = (danger_time_total / seconds_per_year)

    return danger_prob

In [309]:
accident_prob_tops = accident_prob_top(60,4.4,1200)
accident_prob_fronts = accident_prob_front(1.5,1200)

In [310]:
def rockfall_sim(sim_count,df1,df2):
    start_time = time.time()
    param = stats.gamma.fit(df1['Mass_(kg)'])
    zone1_mass = stats.gamma.rvs(*param, size = sim_count)
    param = stats.norm.fit(df1["Velocity_(m/s)"])
    zone1_velocity = stats.norm.rvs(*param, size = sim_count)
    param = stats.expon.fit(df1["Time_delta_(h)"])
    zone1_time_delta = stats.expon.rvs(*param, size = sim_count)
    param = stats.gamma.fit(df2['Mass_(kg)'])
    zone2_mass = stats.gamma.rvs(*param, size = sim_count)
    param = stats.norm.fit(df2["Velocity_(m/s)"])
    zone2_velocity = stats.norm.rvs(*param, size = sim_count)
    param = stats.expon.fit(df2["Time_delta_(h)"])
    zone2_time_delta = stats.expon.rvs(*param, size = sim_count)

    df1_sim = pd.DataFrame()

    df1_sim.insert(0, "Mass_(kg)", zone1_mass)
    df1_sim.insert(1, "Velocity_(m/s)", zone1_velocity)
    df1_sim.insert(2, "Energy_(kJ)", "")
    df1_sim.insert(3, "Time_delta_(h)", zone1_time_delta)
    df1_sim["Energy_(kJ)"] = (0.5 * df1_sim["Mass_(kg)"] * (df1_sim["Velocity_(m/s)"] ** 2)) / 1000
    df1_sim["Time_delta_(h)"] = zone1_time_delta.round(2)
    df1_sim["Time_delta_(h)"][0] = 0
    if 'Time_total' not in df1_sim.columns:
        df1_sim.insert(4, "Time_total", "")
    df2_sim = pd.DataFrame()
    df2_sim.insert(0, "Mass_(kg)", zone2_mass)
    df2_sim.insert(1, "Velocity_(m/s)", zone2_velocity)
    df2_sim.insert(2, "Energy_(kJ)", "")
    df2_sim.insert(3, "Time_delta_(h)", zone2_time_delta)
    df2_sim["Energy_(kJ)"] = (0.5 * df2_sim["Mass_(kg)"] * (df2_sim["Velocity_(m/s)"] ** 2)) / 1000
    df2_sim["Time_delta_(h)"] = zone2_time_delta.round(2)
    df2_sim["Time_delta_(h)"][0] = 0
    if 'Time_total' not in df2_sim.columns:
        df2_sim.insert(4, "Time_total", "")

    df1_sim["Time_total"] = np.cumsum(df1_sim["Time_delta_(h)"])
    df2_sim["Time_total"] = np.cumsum(df2_sim["Time_delta_(h)"])
    df2_sim = df2_sim.loc[df2_sim['Time_total'] <= sum(df1_sim["Time_delta_(h)"])]
    df_sim = pd.concat([df1_sim,df2_sim],ignore_index= True)
    df_sim = df_sim.sort_values(["Time_total"],ignore_index=True)
    df_sim["Time_delta_(h)"] = df_sim["Time_total"].diff()
    df_sim = df_sim.drop("Time_total", axis=1)
    df_sim["Time_delta_(h)"][0] = 0

    time_interval = 24
    current_delta = 0
    hours_total = 0
    weight_in_net = 0
    net_status = False
    incident = 0
    stones_in_net = 0
    sim_stones = len(df_sim)
    for i in range(int(sim_stones)):
        iterated_delta = df_sim["Time_delta_(h)"][i]
        iterated_energy = df_sim["Energy_(kJ)"][i]
        hours_total += iterated_delta
        if (iterated_delta > time_interval):
            if net_status:
                incident += stones_in_net
            stones_in_net = 1
            net_status = net_test(False,0,iterated_energy)
            weight_in_net = df_sim["Mass_(kg)"][i]
            iterated_delta = iterated_delta % time_interval
            if iterated_delta > (time_interval - current_delta):
                current_delta = abs((time_interval - current_delta)-iterated_delta)
            elif iterated_delta <= (time_interval - current_delta):
                current_delta = current_delta + iterated_delta
        elif iterated_delta <= time_interval:
            if iterated_delta > (time_interval - current_delta):
                if net_status:
                    incident += stones_in_net
                stones_in_net = 1
                net_status = net_test(False,0,iterated_energy)
                weight_in_net = df_sim["Mass_(kg)"][i]
                current_delta = abs((time_interval - current_delta)-iterated_delta)
            elif iterated_delta <= (time_interval - current_delta):
                stones_in_net += 1
                net_status = net_test(net_status,weight_in_net,iterated_energy)
                weight_in_net += df_sim["Mass_(kg)"][i]
                current_delta = current_delta + iterated_delta

    if net_status:
        incident += stones_in_net

    amount_sim_years = hours_total / 8760
    stones_per_year = (sim_stones / amount_sim_years)
    incident_prob_per_stone = incident/sim_stones
    incident_prob_per_year = incident_prob_per_stone * stones_per_year
    stone_hit_top_prob = accident_prob_tops * incident_prob_per_year
    stone_hit_front_prob = accident_prob_fronts * incident_prob_per_year
    stone_hit_total_prob = stone_hit_top_prob + stone_hit_front_prob
    final_prob = (stone_hit_total_prob * 1.6)

    end_time = time.time()
    calctime = end_time - start_time
    sys.path.append('c:/users/logan/appdata/local/packages/pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0/localcache/local-packages/python310/site-packages')

    data = {"timestamp": calctime,"sim_count": sim_count, "P_top": stone_hit_top_prob, "P_front": stone_hit_front_prob, "p_netbreak": incident_prob_per_year, "final_prob": final_prob}
    new_row = pd.DataFrame(data, index=[0])
    existing_df = pd.read_excel("Results.xlsx")
    appended_df = existing_df.append(new_row, ignore_index=True)
    appended_df.to_excel("Results.xlsx", index=False)
    return final_prob

In [311]:
def open_csv_to_df(directory):
    df = pd.read_csv(directory)
    df = df.iloc[0:,:4]
    df.columns = ["Date", "Time", "Mass_(kg)", "Velocity_(m/s)"]
    df = df.sort_values(['Date', 'Time']).reset_index(drop=True)
    df = df.dropna()
    df["Date"] = pd.to_datetime(df.Date.astype(str) + " " +  df.Time.astype(str), format = '%Y-%m-%d %H:%M')
    df.rename(columns = {'Date':'Datetime'}, inplace=True)

    return df

In [312]:
def calc_time_delta(df):

    # Neue Spalte erstellen
    df.insert(3, "Time_delta_(h)", "")

    # Zeitabstand berechnen
    for i in range(len(df)-1):
        date1 = df.iloc[i,0]
        date2 = df.iloc[i+1,0]
        time_delta = date2 - date1
        time_delta = (time_delta.days*24) + (time_delta.seconds//3600)
        df.iloc[i+1,3] = time_delta

    # Erster Eintrag mit Null ersetzen
    df.iloc[0,3] = 0

    # Datentyp zu Integer ändern
    df['Time_delta_(h)'] = df['Time_delta_(h)'].astype('int')

    return df

In [313]:
df_area1 = open_csv_to_df("./out_1.csv")
df_area2 = open_csv_to_df("./out_2.csv")
df_area1 = calc_time_delta(df_area1)
df_area2 = calc_time_delta(df_area2)
df_area2['Mass_(kg)'] = df_area2['Mass_(kg)'].replace(0, df_area2['Mass_(kg)'].median())

In [314]:
rockfall_sim(100000,df_area1,df_area2)

C:\Users\logan\AppData\Local\Temp\ipykernel_3012\2389611256.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_df = existing_df.append(new_row, ignore_index=True)


0.0010260634453652627

In [315]:
"""
stone_probs = []
for i in range(10,1000000,10000):
    print(i)
    result = rockfall_sim(i, df_area1, df_area2)
    stone_probs.append((result,i))
"""

'\nstone_probs = []\nfor i in range(10,1000000,10000):\n    print(i)\n    result = rockfall_sim(i, df_area1, df_area2)\n    stone_probs.append((result,i))\n'

In [316]:
"""
for i in range(10_000_000,50_000_000,10_000_000):

    result = rockfall_sim(i, df_area1, df_area2)
    stone_probs.append((result,i))
    print(i)
"""

'\nfor i in range(10_000_000,50_000_000,10_000_000):\n\n    result = rockfall_sim(i, df_area1, df_area2)\n    stone_probs.append((result,i))\n    print(i)\n'

In [317]:
#stone_probs = stone_probs[0:99]

In [318]:
"""
x = [i[1] for i in stone_probs]
y = [i[0] for i in stone_probs]

plt.plot(x, y)
plt.xlabel("i values")
plt.ylabel("result values")
plt.title("stone_probs Plot")
plt.show()
"""

'\nx = [i[1] for i in stone_probs]\ny = [i[0] for i in stone_probs]\n\nplt.plot(x, y)\nplt.xlabel("i values")\nplt.ylabel("result values")\nplt.title("stone_probs Plot")\nplt.show()\n'

In [319]:
def run_sim(sim_loops,sim_count):
    for i in range(0,sim_loops):
        start_time = time.time()
        final_prob = rockfall_sim(sim_count,df_area1,df_area2)



In [ ]:
run_sim(10,10_000_000)

C:\Users\logan\AppData\Local\Temp\ipykernel_3012\2389611256.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_df = existing_df.append(new_row, ignore_index=True)
C:\Users\logan\AppData\Local\Temp\ipykernel_3012\2389611256.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_df = existing_df.append(new_row, ignore_index=True)
C:\Users\logan\AppData\Local\Temp\ipykernel_3012\2389611256.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_df = existing_df.append(new_row, ignore_index=True)
C:\Users\logan\AppData\Local\Temp\ipykernel_3012\2389611256.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appen